# Projeto de Sinais e Sistemas para Computação (SSC)

## III - Algoritmo de Classificação

Este notebook implementa o algoritmo de classificação:
- **Random Forest**

### Objetivos:
- 

In [ ]:
# Imports necessários


## 1. Carregamento e Preparação dos Dados

Carregamos o dataset de características gerado no notebook anterior e preparamos os dados para classificação.

## 2. Pré-processamento dos Dados

Preparamos os dados para os algoritmos de classificação: divisão treino/teste, normalização e codificação de labels.

## 3. Implementação dos Algoritmos de Classificação


In [ ]:
# Configurar modelos com hiperparâmetros otimizados
modelos = {
    '🌲 Random Forest': {
        'estimador': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        ),
        'params_grid': {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10]
        }
    }
}

# Dicionário para armazenar resultados
resultados = {
    'modelo': [],
    'acuracia_cv': [],
    'std_cv': [],
    'acuracia_teste': [],
    'tempo_treino': [],
    'melhor_params': []
}

Um exemplo

In [ ]:
# Loop de treinamento com otimização de hiperparâmetros
    
    # Medir tempo de execução
    inicio = time.time()
    
    # Grid Search com validação cruzada
    try:
        grid_search = GridSearchCV(
            estimator=config['estimador'],
            param_grid=config['params_grid'],
            cv=cv_folds,
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        # Treinar com grid search
        grid_search.fit(X_train_scaled, y_train_encoded)
        
        # Melhor modelo
        melhor_modelo = grid_search.best_estimator_
        
        # Avaliação com validação cruzada
        scores_cv = cross_val_score(
            melhor_modelo, X_train_scaled, y_train_encoded, 
            cv=cv_folds, scoring='accuracy'
        )
        
        # Previsões no conjunto de teste
        y_pred = melhor_modelo.predict(X_test_scaled)
        acuracia_teste = accuracy_score(y_test_encoded, y_pred)
        
        # Tempo de execução
        tempo_execucao = time.time() - inicio
        
        # Armazenar resultados
        resultados['modelo'].append(nome)
        resultados['acuracia_cv'].append(scores_cv.mean())
        resultados['std_cv'].append(scores_cv.std())
        resultados['acuracia_teste'].append(acuracia_teste)
        resultados['tempo_treino'].append(tempo_execucao)
        resultados['melhor_params'].append(grid_search.best_params_)
        
        # Mostrar progresso
        print(f"  ✅ Concluído em {tempo_execucao:.2f}s")
        print(f"  📊 CV Accuracy: {scores_cv.mean():.4f} (±{scores_cv.std():.4f})")
        print(f"  🎯 Test Accuracy: {acuracia_teste:.4f}")
        print(f"  ⚙️ Melhores parâmetros: {grid_search.best_params_}")
        
    except Exception as e:
        print(f" Erro no treinamento: {str(e)}")
        # Adicionar valores padrão em caso de erro
        resultados['modelo'].append(nome)
        resultados['acuracia_cv'].append(0.0)
        resultados['std_cv'].append(0.0)
        resultados['acuracia_teste'].append(0.0)
        resultados['tempo_treino'].append(0.0)
        resultados['melhor_params'].append({})

print(f"\n🏁 Treinamento concluído!")
print("=" * 50)

# Criar DataFrame com resultados
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('acuracia_teste', ascending=False)

## 4. Avaliação Detalhada dos Modelos
Gerar os gráficos (nesse código ele está comparando valores, é só um exemplo)

In [ ]:
# Visualizações comparativas

# 1. Gráfico de barras - Acurácia
axes[0, 0].bar(range(len(df_resultados)), df_resultados['acuracia_teste'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'], alpha=0.8)
axes[0, 0].set_title('🎯 Acurácia no Conjunto de Teste', fontweight='bold')
axes[0, 0].set_ylabel('Acurácia')
axes[0, 0].set_xticks(range(len(df_resultados)))
axes[0, 0].set_xticklabels([m.replace('🌲 ', '').replace('🚀 ', '').replace('📏 ', '').replace('📊 ', '').replace('🧠 ', '') 
                           for m in df_resultados['modelo']], rotation=45, ha='right')
axes[0, 0].grid(True, alpha=0.3)
for i, v in enumerate(df_resultados['acuracia_teste']):
    axes[0, 0].text(i, v + 0.005, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Validação Cruzada com barras de erro
x_pos = range(len(df_resultados))
axes[0, 1].bar(x_pos, df_resultados['acuracia_cv'], 
               yerr=df_resultados['std_cv'], capsize=5,
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'], alpha=0.8)
axes[0, 1].set_title('📊 Validação Cruzada (5-fold)', fontweight='bold')
axes[0, 1].set_ylabel('Acurácia CV')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels([m.replace('🌲 ', '').replace('🚀 ', '').replace('📏 ', '').replace('📊 ', '').replace('🧠 ', '') 
                           for m in df_resultados['modelo']], rotation=45, ha='right')
axes[0, 1].grid(True, alpha=0.3)

# 3. Tempo de treinamento
axes[1, 0].bar(range(len(df_resultados)), df_resultados['tempo_treino'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'], alpha=0.8)
axes[1, 0].set_title('⏱️ Tempo de Treinamento', fontweight='bold')
axes[1, 0].set_ylabel('Tempo (segundos)')
axes[1, 0].set_xticks(range(len(df_resultados)))
axes[1, 0].set_xticklabels([m.replace('🌲 ', '').replace('🚀 ', '').replace('📏 ', '').replace('📊 ', '').replace('🧠 ', '') 
                           for m in df_resultados['modelo']], rotation=45, ha='right')
axes[1, 0].grid(True, alpha=0.3)
for i, v in enumerate(df_resultados['tempo_treino']):
    axes[1, 0].text(i, v + max(df_resultados['tempo_treino'])*0.01, f'{v:.1f}s', 
                    ha='center', va='bottom', fontweight='bold')

# 4. Comparação Acurácia vs Tempo
scatter = axes[1, 1].scatter(df_resultados['tempo_treino'], df_resultados['acuracia_teste'], 
                            s=200, alpha=0.7, c=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
axes[1, 1].set_title('⚖️ Acurácia vs Tempo de Treinamento', fontweight='bold')
axes[1, 1].set_xlabel('Tempo de Treinamento (s)')
axes[1, 1].set_ylabel('Acurácia')
axes[1, 1].grid(True, alpha=0.3)

# Adicionar labels aos pontos
for i, (x, y, nome) in enumerate(zip(df_resultados['tempo_treino'], 
                                    df_resultados['acuracia_teste'], 
                                    df_resultados['modelo'])):
    nome_curto = nome.replace('🌲 ', '').replace('🚀 ', '').replace('📏 ', '').replace('📊 ', '').replace('🧠 ', '')
    axes[1, 1].annotate(nome_curto, (x, y), xytext=(5, 5), textcoords='offset points',
                       fontsize=9, ha='left')

plt.tight_layout()
plt.show()

# Tabela resumo
print("📋 TABELA RESUMO DOS RESULTADOS:")
print("=" * 80)
print(f"{'Modelo':<20} {'Acurácia CV':<12} {'Std CV':<10} {'Acurácia Teste':<15} {'Tempo (s)':<10}")
print("-" * 80)
for _, row in df_resultados.iterrows():
    nome_limpo = row['modelo'].replace('🌲 ', '').replace('🚀 ', '').replace('📏 ', '').replace('📊 ', '').replace('🧠 ', '')
    print(f"{nome_limpo:<20} {row['acuracia_cv']:<12.4f} {row['std_cv']:<10.4f} {row['acuracia_teste']:<15.4f} {row['tempo_treino']:<10.2f}")

print("\n🏆 MELHOR MODELO:", df_resultados.iloc[0]['modelo'])
print(f"   🎯 Acurácia: {df_resultados.iloc[0]['acuracia_teste']:.4f}")
print(f"   ⚙️  Parâmetros: {df_resultados.iloc[0]['melhor_params']}")

In [ ]:
# Análise detalhada do melhor modelo
melhor_modelo_nome = df_resultados.iloc[0]['modelo']
melhor_params = df_resultados.iloc[0]['melhor_params']

print(f"🔍 ANÁLISE DETALHADA - {melhor_modelo_nome}")
print("=" * 60)

# Retreinar o melhor modelo para análise detalhada
melhor_config = modelos[melhor_modelo_nome]
modelo_final = melhor_config['estimador'].set_params(**melhor_params)
modelo_final.fit(X_train_scaled, y_train_encoded)

# Previsões
y_pred_final = modelo_final.predict(X_test_scaled)
y_pred_proba = modelo_final.predict_proba(X_test_scaled) if hasattr(modelo_final, 'predict_proba') else None

# Matriz de confusão
cm = confusion_matrix(y_test_encoded, y_pred_final)
class_names = label_encoder.classes_

# Visualização da matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Número de Amostras'})
plt.title(f'🎯 Matriz de Confusão - {melhor_modelo_nome}', fontsize=14, fontweight='bold')
plt.xlabel('Classe Predita', fontweight='bold')
plt.ylabel('Classe Real', fontweight='bold')
plt.tight_layout()
plt.show()

# Relatório de classificação
print(f"\n📊 RELATÓRIO DE CLASSIFICAÇÃO:")
print("=" * 50)
report = classification_report(y_test_encoded, y_pred_final, 
                             target_names=class_names, output_dict=True)

# Exibir métricas por classe
for classe in class_names:
    metrics = report[classe]
    print(f"\n🏷️  Classe '{classe}':")
    print(f"   Precisão:  {metrics['precision']:.4f}")
    print(f"   Recall:    {metrics['recall']:.4f}")
    print(f"   F1-Score:  {metrics['f1-score']:.4f}")
    print(f"   Suporte:   {metrics['support']:.0f} amostras")

# Métricas globais
print(f"\n🌐 MÉTRICAS GLOBAIS:")
print(f"   Acurácia:     {report['accuracy']:.4f}")
print(f"   Macro Avg:    {report['macro avg']['f1-score']:.4f}")
print(f"   Weighted Avg: {report['weighted avg']['f1-score']:.4f}")

# Análise de erro por classe
print(f"\n❌ ANÁLISE DE ERROS:")
print("=" * 30)
for i, classe_real in enumerate(class_names):
    for j, classe_pred in enumerate(class_names):
        if i != j and cm[i, j] > 0:
            taxa_erro = cm[i, j] / cm[i, :].sum() * 100
            print(f"   {classe_real} → {classe_pred}: {cm[i, j]} erros ({taxa_erro:.1f}%)")

# Calcular métricas adicionais
print(f"\n📈 MÉTRICAS ADICIONAIS:")
print("=" * 25)

# Precisão balanceada
balanced_acc = balanced_accuracy_score(y_test_encoded, y_pred_final)
print(f"   Acurácia Balanceada: {balanced_acc:.4f}")

# Kappa de Cohen
kappa = cohen_kappa_score(y_test_encoded, y_pred_final)
print(f"   Kappa de Cohen: {kappa:.4f}")

# MCC (Matthews Correlation Coefficient) - apenas para classificação binária
if len(class_names) == 2:
    mcc = matthews_corrcoef(y_test_encoded, y_pred_final)
    print(f"   Matthews Correlation: {mcc:.4f}")

print(f"\n✅ Análise do melhor modelo concluída!")

## 5. Conclusões

### 🎯 Principais Descobertas

1. **Trade-offs**:
2. **Robustez**:
3. **Otimização**:

### 📊 Métricas Importantes

- 


### 💾 Salvamento do Modelo

O melhor modelo será salvo para uso futuro e implementação em produção.